In [41]:
import pandas as pd
import requests
import time
import geopandas as gpd
from dateutil.relativedelta import relativedelta
import datetime as dt
import math

In [42]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [43]:
# Read in new file
file_loc_raw = "../datasets/resale_hdb_price_raw_31jan25.csv"
df_raw = pd.read_csv(file_loc_raw, parse_dates=["month", "lease_commence_date"])

df_raw["floor_area_sqft"] = df_raw["floor_area_sqm"] * 10.7639
df_raw["price_per_sqft"] = (df_raw["resale_price"] / df_raw["floor_area_sqft"])

# df_raw = df_raw.drop(["remaining_lease", "lease_commence_date"], axis=1)

In [44]:
df_raw.head(1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,473.6116,489.852867


In [45]:
# df_raw = df_raw.query("month != '2025-01-01'")

In [46]:
df_raw.shape

(199327, 13)

### Begin the synchronisation process

#### 1. Open the masterlist of HDB addresses. This will be used to perform a left join to the new data

In [47]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc, index_col=0, dtype={"postal": "object"}, parse_dates=["lease_commence_date"]
)

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.052240,38692.616791,1.366197,103.841505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9659,YISHUN,876,YISHUN ST 81,876,YISHUN STREET 81,NIL,760876,876 YISHUN STREET 81 SINGAPORE 760876,1987-12-01,YISHUN,NORTH REGION,28246.274546,44060.806242,1.414745,103.835532
9660,YISHUN,877,YISHUN ST 81,877,YISHUN STREET 81,NIL,760877,877 YISHUN STREET 81 SINGAPORE 760877,1987-12-01,YISHUN,NORTH REGION,28237.634702,43967.636908,1.413902,103.835454
9661,YISHUN,878,YISHUN ST 81,878,YISHUN STREET 81,NIL,760878,878 YISHUN STREET 81 SINGAPORE 760878,1988-01-01,YISHUN,NORTH REGION,28285.898091,43984.300584,1.414053,103.835888
9662,YISHUN,879,YISHUN ST 81,879,YISHUN STREET 81,NIL,760879,879 YISHUN STREET 81 SINGAPORE 760879,1987-10-01,YISHUN,NORTH REGION,28311.512736,44027.290776,1.414442,103.836118


#### 1a: First, check if there are any new addresses that appear in the new dataframe

In [8]:
df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge
198340,2025-01-01,QUEENSTOWN,4 ROOM,78,DAWSON RD,25 TO 27,88.0,Model A,2020-01-01,94 years 11 months,1225000.0,947.2232,1293.253797,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198374,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,01 TO 03,47.0,2-room,2020-01-01,95 years,333000.0,505.9033,658.228559,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198378,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,10 TO 12,38.0,2-room,2020-01-01,95 years,359000.0,409.0282,877.690096,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198379,2025-01-01,SEMBAWANG,2 ROOM,103B,CANBERRA ST,10 TO 12,38.0,2-room,2020-01-01,95 years,330000.0,409.0282,806.790339,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198422,2025-01-01,SEMBAWANG,4 ROOM,126D,CANBERRA ST,13 TO 15,96.0,Model A,2020-01-01,94 years 10 months,730000.0,1033.3344,706.450884,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198495,2025-01-01,SENGKANG,2 ROOM,355A,ANCHORVALE LANE,04 TO 06,47.0,2-room,2021-01-01,95 years 02 months,392000.0,505.9033,774.851637,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198726,2025-01-01,TAMPINES,3 ROOM,613A,TAMPINES NTH DR 1,13 TO 15,69.0,Model A,2020-01-01,94 years 10 months,650000.0,742.7091,875.174412,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198784,2025-01-01,TAMPINES,4 ROOM,613A,TAMPINES NTH DR 1,10 TO 12,93.0,Model A,2020-01-01,94 years 10 months,780000.0,1001.0427,779.187541,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
198789,2025-01-01,TAMPINES,4 ROOM,613A,TAMPINES NTH DR 1,04 TO 06,93.0,Model A,2020-01-01,94 years 10 months,750000.0,1001.0427,749.218790,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


#### 1b: If there are new addresses, use the following code to select the correct address, and insert ancillary information like floor area in square feet, price per square foot, and recalculating lease commence date

In [9]:
# First, check for any missing values in the masterlist
missing_val = masterlist_df[masterlist_df.isna().any(axis=1)]
missing_val

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters


In [10]:
# Obtain the index of data points whose block + street name + postal + building information aren't present in the masterlist
for_editing_index = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'").drop_duplicates(
    subset=["town", "block", "street_name"]
).index

for_editing = df_raw.loc[for_editing_index, :]
for_editing.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft
198340,2025-01-01,QUEENSTOWN,4 ROOM,78,DAWSON RD,25 TO 27,88.0,Model A,2020-01-01,94 years 11 months,1225000.0,947.2232,1293.253797
198374,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,01 TO 03,47.0,2-room,2020-01-01,95 years,333000.0,505.9033,658.228559
198379,2025-01-01,SEMBAWANG,2 ROOM,103B,CANBERRA ST,10 TO 12,38.0,2-room,2020-01-01,95 years,330000.0,409.0282,806.790339
198422,2025-01-01,SEMBAWANG,4 ROOM,126D,CANBERRA ST,13 TO 15,96.0,Model A,2020-01-01,94 years 10 months,730000.0,1033.3344,706.450884
198495,2025-01-01,SENGKANG,2 ROOM,355A,ANCHORVALE LANE,04 TO 06,47.0,2-room,2021-01-01,95 years 02 months,392000.0,505.9033,774.851637


In [11]:
def obtain_lease_yearmth(row):
    today = pd.to_datetime(dt.date.today().replace(day=1))
    lease_commence = today - pd.DateOffset(
        years=row["lease_year"] + 99, months=row["lease_month"]
    )
    return lease_commence


for_editing["lease_year"] = for_editing["remaining_lease"].str.slice(0, 2).astype("int")
for_editing["lease_month"] = pd.to_numeric(
    for_editing["remaining_lease"].str.slice(9, 11), errors="coerce"
).fillna(0)

for_editing["lease_commence_date"] = for_editing.apply(obtain_lease_yearmth, axis=1)
for_editing = for_editing.drop(
    ["remaining_lease", "lease_year", "lease_month"], axis=1
)

In [12]:
for_editing.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft
198340,2025-01-01,QUEENSTOWN,4 ROOM,78,DAWSON RD,25 TO 27,88.0,Model A,1831-03-01,1225000.0,947.2232,1293.253797
198374,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,01 TO 03,47.0,2-room,1831-02-01,333000.0,505.9033,658.228559
198379,2025-01-01,SEMBAWANG,2 ROOM,103B,CANBERRA ST,10 TO 12,38.0,2-room,1831-02-01,330000.0,409.0282,806.790339
198422,2025-01-01,SEMBAWANG,4 ROOM,126D,CANBERRA ST,13 TO 15,96.0,Model A,1831-04-01,730000.0,1033.3344,706.450884
198495,2025-01-01,SENGKANG,2 ROOM,355A,ANCHORVALE LANE,04 TO 06,47.0,2-room,1830-12-01,392000.0,505.9033,774.851637


In [15]:
if for_editing.shape[0] > 0:
    for i in for_editing.index:
        search_value = (
            for_editing.loc[i, "block"] + " " + for_editing.loc[i, "street_name"]
        )
        print(f"{i}: {search_value}")
        response = requests.get(
            f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        )
        while response.status_code != 200:
            time.sleep(1)
            print("not 200")
            response = requests.get(
                f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
            )

        json_data = response.json()
        for j in json_data["results"]:
            print(j)
        time.sleep(0.5)
        user_input = input("Select the correct address: ")
        k = int(user_input) - 1
        print(k)

        if k == -2:
            for_editing.loc[i, "found"] = math.nan
            for_editing.loc[i, "search_val"] = math.nan
            for_editing.loc[i, "blk_no"] = math.nan
            for_editing.loc[i, "road_name"] = math.nan
            for_editing.loc[i, "building"] = math.nan
            for_editing.loc[i, "address"] = math.nan
            for_editing.loc[i, "postal"] = math.nan
            for_editing.loc[i, "x"] = math.nan
            for_editing.loc[i, "y"] = math.nan
            for_editing.loc[i, "latitude"] = math.nan
            for_editing.loc[i, "longitude"] = math.nan
        else:
            for_editing.loc[i, "found"] = 1
            for_editing.loc[i, "search_val"] = json_data["results"][k]["SEARCHVAL"]
            for_editing.loc[i, "blk_no"] = json_data["results"][k]["BLK_NO"]
            for_editing.loc[i, "road_name"] = json_data["results"][k]["ROAD_NAME"]
            for_editing.loc[i, "building"] = json_data["results"][k]["BUILDING"]
            for_editing.loc[i, "address"] = json_data["results"][k]["ADDRESS"]
            for_editing.loc[i, "postal"] = json_data["results"][k]["POSTAL"]
            for_editing.loc[i, "x"] = json_data["results"][k]["X"]
            for_editing.loc[i, "y"] = json_data["results"][k]["Y"]
            for_editing.loc[i, "latitude"] = json_data["results"][k]["LATITUDE"]
            for_editing.loc[i, "longitude"] = json_data["results"][k]["LONGITUDE"]

198340: 78 DAWSON RD
{'SEARCHVAL': 'DAWSON VISTA', 'BLK_NO': '78', 'ROAD_NAME': 'DAWSON ROAD', 'BUILDING': 'DAWSON VISTA', 'ADDRESS': '78 DAWSON ROAD DAWSON VISTA SINGAPORE 141078', 'POSTAL': '141078', 'X': '25415.1916509736', 'Y': '30694.5215664282', 'LATITUDE': '1.29386475464004', 'LONGITUDE': '103.810092969298'}
0
198374: 103A CANBERRA ST
{'SEARCHVAL': 'EASTCREEK @ CANBERRA', 'BLK_NO': '103A', 'ROAD_NAME': 'CANBERRA STREET', 'BUILDING': 'EASTCREEK @ CANBERRA', 'ADDRESS': '103A CANBERRA STREET EASTCREEK @ CANBERRA SINGAPORE 751103', 'POSTAL': '751103', 'X': '27690.1218658667', 'Y': '48073.0501046516', 'LATITUDE': '1.45103006132424', 'LONGITUDE': '103.830534008712'}
0
198379: 103B CANBERRA ST
{'SEARCHVAL': 'EASTCREEK @ CANBERRA', 'BLK_NO': '103B', 'ROAD_NAME': 'CANBERRA STREET', 'BUILDING': 'EASTCREEK @ CANBERRA', 'ADDRESS': '103B CANBERRA STREET EASTCREEK @ CANBERRA SINGAPORE 752103', 'POSTAL': '752103', 'X': '27768.5473315939', 'Y': '48021.7358535491', 'LATITUDE': '1.45056599465441'

In [16]:
for_editing_gdf = gpd.GeoDataFrame(
    for_editing, geometry=gpd.points_from_xy(for_editing["x"], for_editing["y"])
)

for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry
198340,2025-01-01,QUEENSTOWN,4 ROOM,78,DAWSON RD,25 TO 27,88.0,Model A,1831-03-01,1225000.0,947.2232,1293.253797,1.0,DAWSON VISTA,78,DAWSON ROAD,DAWSON VISTA,78 DAWSON ROAD DAWSON VISTA SINGAPORE 141078,141078,25415.1916509736,30694.5215664282,1.29386475464004,103.810092969298,POINT (25415.192 30694.522)
198374,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,01 TO 03,47.0,2-room,1831-02-01,333000.0,505.9033,658.228559,1.0,EASTCREEK @ CANBERRA,103A,CANBERRA STREET,EASTCREEK @ CANBERRA,103A CANBERRA STREET EASTCREEK @ CANBERRA SING...,751103,27690.1218658667,48073.0501046516,1.45103006132424,103.830534008712,POINT (27690.122 48073.050)
198379,2025-01-01,SEMBAWANG,2 ROOM,103B,CANBERRA ST,10 TO 12,38.0,2-room,1831-02-01,330000.0,409.0282,806.790339,1.0,EASTCREEK @ CANBERRA,103B,CANBERRA STREET,EASTCREEK @ CANBERRA,103B CANBERRA STREET EASTCREEK @ CANBERRA SING...,752103,27768.5473315939,48021.7358535491,1.45056599465441,103.831238741555,POINT (27768.547 48021.736)
198422,2025-01-01,SEMBAWANG,4 ROOM,126D,CANBERRA ST,13 TO 15,96.0,Model A,1831-04-01,730000.0,1033.3344,706.450884,1.0,EASTDELTA @ CANBERRA,126D,CANBERRA STREET,EASTDELTA @ CANBERRA,126D CANBERRA STREET EASTDELTA @ CANBERRA SING...,754126,28164.0242061103,47580.4346440118,1.4465750273277,103.834792499436,POINT (28164.024 47580.435)
198495,2025-01-01,SENGKANG,2 ROOM,355A,ANCHORVALE LANE,04 TO 06,47.0,2-room,1830-12-01,392000.0,505.9033,774.851637,1.0,ANCHORVALE PLAINS,355A,ANCHORVALE LANE,ANCHORVALE PLAINS,355A ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,541355,33763.3906799816,41652.9829311017,1.39296872192969,103.885107193078,POINT (33763.391 41652.983)
198726,2025-01-01,TAMPINES,3 ROOM,613A,TAMPINES NTH DR 1,13 TO 15,69.0,Model A,1831-04-01,650000.0,742.7091,875.174412,1.0,TAMPINES GREENVIEW,613A,TAMPINES NORTH DRIVE 1,TAMPINES GREENVIEW,613A TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,521613,39351.869820646,38648.2615424097,1.3657934916842,103.935322892026,POINT (39351.870 38648.262)


#### 1c. Appending MRT information

In [19]:
mrt_file_loc = "../datasets/mrt_lrt_stations_2025-01-14.csv"
mrt_df = pd.read_csv(mrt_file_loc, parse_dates=["opening"], index_col=0)
mrt_gdf = gpd.GeoDataFrame(mrt_df, geometry=gpd.points_from_xy(mrt_df["x"], mrt_df["y"]))

mrt_gdf.head(1)

,code,station_name,line,color,opening,type,blk_no,road_name,building,address,postal,x,y,latitude,longitude,planning_area_ura,region_ura,geometry
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.057052,35038.96887,1.333153,103.742286,JURONG EAST,WEST REGION,POINT (17869.057 35038.969)


In [20]:
def find_closest_station(row, mrt_gdf):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # mrt_gdf["OPENING_DATE"] = mrt_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = mrt_gdf[mrt_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    """Remove comments for this line if you want the closest MRT station today"""
    distances = mrt_gdf.distance(row["geometry"])

    closest_station_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_station_name = mrt_gdf.loc[closest_station_index, "station_name"]
    closest_transport_type = mrt_gdf.loc[closest_station_index, "type"]
    closest_mrt_color = mrt_gdf.loc[closest_station_index, "color"]

    # distance to cbd
    raffles_place_index = mrt_gdf.query("station_name == 'Raffles Place'").index[0]
    distance_to_cbd = mrt_gdf.loc[raffles_place_index, "geometry"].distance(
        row["geometry"]
    )

    return pd.Series(
        [
            closest_station_name,
            shortest_distance,
            closest_transport_type,
            closest_mrt_color,
            distance_to_cbd,
        ],
        index=[
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ],
    )


if for_editing_gdf.shape[0] > 0:
    for_editing_gdf[
        [
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ]
    ] = for_editing_gdf.apply(find_closest_station, mrt_gdf=mrt_gdf, axis=1)

In [21]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd
198340,2025-01-01,QUEENSTOWN,4 ROOM,78,DAWSON RD,25 TO 27,88.0,Model A,1831-03-01,1225000.0,947.2232,1293.253797,1.0,DAWSON VISTA,78,DAWSON ROAD,DAWSON VISTA,78 DAWSON ROAD DAWSON VISTA SINGAPORE 141078,141078,25415.1916509736,30694.5215664282,1.29386475464004,103.810092969298,POINT (25415.192 30694.522),Queenstown,453.318345,MRT,Green,4728.260413
198374,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,01 TO 03,47.0,2-room,1831-02-01,333000.0,505.9033,658.228559,1.0,EASTCREEK @ CANBERRA,103A,CANBERRA STREET,EASTCREEK @ CANBERRA,103A CANBERRA STREET EASTCREEK @ CANBERRA SING...,751103,27690.1218658667,48073.0501046516,1.45103006132424,103.830534008712,POINT (27690.122 48073.050),Canberra,884.303000,MRT,Red,18601.819489
198379,2025-01-01,SEMBAWANG,2 ROOM,103B,CANBERRA ST,10 TO 12,38.0,2-room,1831-02-01,330000.0,409.0282,806.790339,1.0,EASTCREEK @ CANBERRA,103B,CANBERRA STREET,EASTCREEK @ CANBERRA,103B CANBERRA STREET EASTCREEK @ CANBERRA SING...,752103,27768.5473315939,48021.7358535491,1.45056599465441,103.831238741555,POINT (27768.547 48021.736),Canberra,845.595392,MRT,Red,18541.227119
198422,2025-01-01,SEMBAWANG,4 ROOM,126D,CANBERRA ST,13 TO 15,96.0,Model A,1831-04-01,730000.0,1033.3344,706.450884,1.0,EASTDELTA @ CANBERRA,126D,CANBERRA STREET,EASTDELTA @ CANBERRA,126D CANBERRA STREET EASTDELTA @ CANBERRA SING...,754126,28164.0242061103,47580.4346440118,1.4465750273277,103.834792499436,POINT (28164.024 47580.435),Canberra,685.915599,MRT,Red,18058.365964
198495,2025-01-01,SENGKANG,2 ROOM,355A,ANCHORVALE LANE,04 TO 06,47.0,2-room,1830-12-01,392000.0,505.9033,774.851637,1.0,ANCHORVALE PLAINS,355A,ANCHORVALE LANE,ANCHORVALE PLAINS,355A ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,541355,33763.3906799816,41652.9829311017,1.39296872192969,103.885107193078,POINT (33763.391 41652.983),Tongkang,408.680926,LRT,Grey,12604.334534
198726,2025-01-01,TAMPINES,3 ROOM,613A,TAMPINES NTH DR 1,13 TO 15,69.0,Model A,1831-04-01,650000.0,742.7091,875.174412,1.0,TAMPINES GREENVIEW,613A,TAMPINES NORTH DRIVE 1,TAMPINES GREENVIEW,613A TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,521613,39351.869820646,38648.2615424097,1.3657934916842,103.935322892026,POINT (39351.870 38648.262),Tampines,1454.934341,MRT,Blue,12986.597323


#### 1d. Appending Closest School information

In [22]:
school_file_loc = "../datasets/schools_for_plotly.csv"
school_df = pd.read_csv(school_file_loc, index_col=0, dtype={"postal":"string"})
school_df["postal"] = school_df["postal"].astype("str").apply(lambda x: f"{x:0>6}")

# Convert the df into a gdf
school_gdf = gpd.GeoDataFrame(school_df, geometry=gpd.points_from_xy(school_df["x"], school_df["y"]))

In [23]:
def find_closest_school(row, school_gdf, level="PRIMARY"):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # school_gdf["OPENING_DATE"] = school_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = school_gdf[school_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    school_gdf_filtered = school_gdf.query("mainlevel_code == @level")
    """Remove comments for this line if you want the closest MRT station today"""
    distances = school_gdf_filtered.distance(row["geometry"])

    closest_school_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_school = school_gdf_filtered.loc[closest_school_index, "school_name"]

    return pd.Series(
        [
            closest_school,
            shortest_distance,
        ],
        index=[
            "closest_pri_school",
            "distance_to_pri_school_meters",
        ],
    )

for_editing_gdf[["closest_pri_school", "distance_to_pri_school_meters"]] = (
    for_editing_gdf.apply(find_closest_school, school_gdf=school_gdf, axis=1)
)

In [24]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
198340,2025-01-01,QUEENSTOWN,4 ROOM,78,DAWSON RD,25 TO 27,88.0,Model A,1831-03-01,1225000.0,947.2232,1293.253797,1.0,DAWSON VISTA,78,DAWSON ROAD,DAWSON VISTA,78 DAWSON ROAD DAWSON VISTA SINGAPORE 141078,141078,25415.1916509736,30694.5215664282,1.29386475464004,103.810092969298,POINT (25415.192 30694.522),Queenstown,453.318345,MRT,Green,4728.260413,QUEENSTOWN PRIMARY SCHOOL,341.655988
198374,2025-01-01,SEMBAWANG,2 ROOM,103A,CANBERRA ST,01 TO 03,47.0,2-room,1831-02-01,333000.0,505.9033,658.228559,1.0,EASTCREEK @ CANBERRA,103A,CANBERRA STREET,EASTCREEK @ CANBERRA,103A CANBERRA STREET EASTCREEK @ CANBERRA SING...,751103,27690.1218658667,48073.0501046516,1.45103006132424,103.830534008712,POINT (27690.122 48073.050),Canberra,884.303000,MRT,Red,18601.819489,WELLINGTON PRIMARY SCHOOL,921.173098
198379,2025-01-01,SEMBAWANG,2 ROOM,103B,CANBERRA ST,10 TO 12,38.0,2-room,1831-02-01,330000.0,409.0282,806.790339,1.0,EASTCREEK @ CANBERRA,103B,CANBERRA STREET,EASTCREEK @ CANBERRA,103B CANBERRA STREET EASTCREEK @ CANBERRA SING...,752103,27768.5473315939,48021.7358535491,1.45056599465441,103.831238741555,POINT (27768.547 48021.736),Canberra,845.595392,MRT,Red,18541.227119,WELLINGTON PRIMARY SCHOOL,1005.537102
198422,2025-01-01,SEMBAWANG,4 ROOM,126D,CANBERRA ST,13 TO 15,96.0,Model A,1831-04-01,730000.0,1033.3344,706.450884,1.0,EASTDELTA @ CANBERRA,126D,CANBERRA STREET,EASTDELTA @ CANBERRA,126D CANBERRA STREET EASTDELTA @ CANBERRA SING...,754126,28164.0242061103,47580.4346440118,1.4465750273277,103.834792499436,POINT (28164.024 47580.435),Canberra,685.915599,MRT,Red,18058.365964,CHONGFU SCHOOL,1034.721464
198495,2025-01-01,SENGKANG,2 ROOM,355A,ANCHORVALE LANE,04 TO 06,47.0,2-room,1830-12-01,392000.0,505.9033,774.851637,1.0,ANCHORVALE PLAINS,355A,ANCHORVALE LANE,ANCHORVALE PLAINS,355A ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,541355,33763.3906799816,41652.9829311017,1.39296872192969,103.885107193078,POINT (33763.391 41652.983),Tongkang,408.680926,LRT,Grey,12604.334534,ANCHOR GREEN PRIMARY SCHOOL,367.472893
198726,2025-01-01,TAMPINES,3 ROOM,613A,TAMPINES NTH DR 1,13 TO 15,69.0,Model A,1831-04-01,650000.0,742.7091,875.174412,1.0,TAMPINES GREENVIEW,613A,TAMPINES NORTH DRIVE 1,TAMPINES GREENVIEW,613A TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,521613,39351.869820646,38648.2615424097,1.3657934916842,103.935322892026,POINT (39351.870 38648.262),Tampines,1454.934341,MRT,Blue,12986.597323,ANGSANA PRIMARY SCHOOL,469.715435


#### 1e. Appending URA planning area and regions

In [25]:
geo_file_loc = "../datasets/sg_map/mp2014/MP14_PLNG_AREA_NO_SEA_PL.shp"
planning_areas_gdf = gpd.read_file(geo_file_loc)

planning_areas_gdf.head()

,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,ANG MO KIO,AM,N,NORTH-EAST REGION,NER,E5CBDDE0C2113055,2016-05-11,28976.8763,40229.1238,17494.240190,1.394138e+07,"POLYGON ((30658.500 42047.527, 30679.195 42020..."
1,2,BEDOK,BD,N,EAST REGION,ER,1719251260799DF6,2016-05-11,38582.6650,34032.0961,21872.798962,2.173319e+07,"POLYGON ((38974.269 36138.243, 39371.471 35747..."
2,3,BISHAN,BS,N,CENTRAL REGION,CR,BA616285F402846F,2016-05-11,28789.7630,37450.8865,13517.121556,7.618921e+06,"POLYGON ((29772.191 38311.805, 29784.826 38304..."
3,4,BOON LAY,BL,N,WEST REGION,WR,A3DC87118B43CDED,2016-05-11,13410.3824,33008.9884,18528.467448,8.279408e+06,"POLYGON ((12861.383 32207.492, 12860.555 32208..."
4,5,BUKIT BATOK,BK,N,WEST REGION,WR,FB44C870B04B7F57,2016-05-11,19255.4150,37527.6527,15234.223423,1.113326e+07,"POLYGON ((20294.455 39114.528, 20334.318 39054..."


In [26]:
for_editing_gdf = for_editing_gdf.sjoin(planning_areas_gdf[["PLN_AREA_N", "REGION_N", "geometry"]], how='left')

/Users/tim/miniforge3/envs/for_ml/lib/python3.10/site-packages/geopandas/geodataframe.py:2189: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: PROJCS["SVY21",GEOGCS["SVY21[WGS84]",DATUM["WGS_19 ...

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


In [27]:
for_editing_gdf = for_editing_gdf.rename(columns={"PLN_AREA_N":"planning_area_ura", "REGION_N":"region_ura"})

In [28]:
for_editing = for_editing_gdf[
    [
        "town",
        "block",
        "road_name",
        "blk_no",
        "street_name",
        "building",
        "postal",
        "address",
        "lease_commence_date",
        "planning_area_ura",
        "region_ura",
        "x",
        "y",
        "latitude",
        "longitude",
        "closest_mrt_station",
        "distance_to_mrt_meters",
        "transport_type",
        "line_color",
        "distance_to_cbd",
        "closest_pri_school",
        "distance_to_pri_school_meters",
    ]
]

In [29]:
for_editing

,town,block,road_name,blk_no,street_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
198340,QUEENSTOWN,78,DAWSON ROAD,78,DAWSON RD,DAWSON VISTA,141078,78 DAWSON ROAD DAWSON VISTA SINGAPORE 141078,1831-03-01,QUEENSTOWN,CENTRAL REGION,25415.1916509736,30694.5215664282,1.29386475464004,103.810092969298,Queenstown,453.318345,MRT,Green,4728.260413,QUEENSTOWN PRIMARY SCHOOL,341.655988
198374,SEMBAWANG,103A,CANBERRA STREET,103A,CANBERRA ST,EASTCREEK @ CANBERRA,751103,103A CANBERRA STREET EASTCREEK @ CANBERRA SING...,1831-02-01,SEMBAWANG,NORTH REGION,27690.1218658667,48073.0501046516,1.45103006132424,103.830534008712,Canberra,884.303000,MRT,Red,18601.819489,WELLINGTON PRIMARY SCHOOL,921.173098
198379,SEMBAWANG,103B,CANBERRA STREET,103B,CANBERRA ST,EASTCREEK @ CANBERRA,752103,103B CANBERRA STREET EASTCREEK @ CANBERRA SING...,1831-02-01,SEMBAWANG,NORTH REGION,27768.5473315939,48021.7358535491,1.45056599465441,103.831238741555,Canberra,845.595392,MRT,Red,18541.227119,WELLINGTON PRIMARY SCHOOL,1005.537102
198422,SEMBAWANG,126D,CANBERRA STREET,126D,CANBERRA ST,EASTDELTA @ CANBERRA,754126,126D CANBERRA STREET EASTDELTA @ CANBERRA SING...,1831-04-01,SEMBAWANG,NORTH REGION,28164.0242061103,47580.4346440118,1.4465750273277,103.834792499436,Canberra,685.915599,MRT,Red,18058.365964,CHONGFU SCHOOL,1034.721464
198495,SENGKANG,355A,ANCHORVALE LANE,355A,ANCHORVALE LANE,ANCHORVALE PLAINS,541355,355A ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,1830-12-01,SENGKANG,NORTH-EAST REGION,33763.3906799816,41652.9829311017,1.39296872192969,103.885107193078,Tongkang,408.680926,LRT,Grey,12604.334534,ANCHOR GREEN PRIMARY SCHOOL,367.472893
198726,TAMPINES,613A,TAMPINES NORTH DRIVE 1,613A,TAMPINES NTH DR 1,TAMPINES GREENVIEW,521613,613A TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,1831-04-01,TAMPINES,EAST REGION,39351.869820646,38648.2615424097,1.3657934916842,103.935322892026,Tampines,1454.934341,MRT,Blue,12986.597323,ANGSANA PRIMARY SCHOOL,469.715435


In [30]:
new_masterlist_df = pd.concat([masterlist_df, for_editing], axis=0)
new_masterlist_df = new_masterlist_df.sort_values(by=["town", "street_name", "block"]).reset_index(drop=True)

In [31]:
new_masterlist_df.shape

(9664, 22)

In [33]:
# new_masterlist_df.to_csv(masterlist_file_loc)

### 2. Append additional information to the raw dataset

#### 2a: Reopen Masterlist

In [35]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc,
    index_col=0,
    dtype={"postal": "object"},
    parse_dates=["lease_commence_date"],
)

masterlist_df.head(1)

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254


In [36]:
masterlist_df.shape

(9664, 22)

#### 2b: Perform a left merge between the new HDB dataset and the address masterlist

In [37]:
df_new = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
)

In [38]:
df_new.query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge


In [39]:
df_new = df_new.drop("_merge", axis=1)

In [40]:
file_destination = file_loc_raw.replace("raw", "coords_mrt")
file_destination
df_new.to_csv(file_destination)

#### The following is used to call the Onemap API and populate coordinate details (deprecated, use the above masterlist instead)

In [ ]:
# # Obtain geospatial coordinates with the Onemap API

# def get_coordinates_for_each_row(row):
#     search_value = row["block"] + " " + row["street_name"]
#     # print(search_value)

#     response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")
#     while response.status_code != 200:
#         time.sleep(1)
#         print("not 200")
#         response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")

#     data = response.json()

#     row_data = data['results'][0]
#     searchval = row_data["SEARCHVAL"]
#     address = row_data["ADDRESS"]
#     postal = row_data["POSTAL"]
#     x = row_data["X"]
#     y = row_data["Y"]
#     latitude = row_data["LATITUDE"]
#     longitude = row_data["LONGITUDE"]

#     return pd.Series([address, postal, x, y, latitude, longitude], 
#                      index=["SEARCHVAL", "ADDRESS", "POSTAL", "X", "Y", "LATITUDE", "LONGITUDE"])

In [ ]:
# Appending additional columns to the dataframe


# NOTE: SHOULDNT APPEND
# for_addition["year"] = for_addition["month"].dt.year
# for_addition["lease_years"] = for_addition["remaining_lease"].str.split(" ").apply(lambda x: int(x[0]))
# bins = pd.IntervalIndex.from_tuples(
#     [(40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
# )
# for_addition["lease_cat"] = pd.cut(for_addition["lease_years"], bins)
# for_addition = for_addition.drop("index", axis=1)

#### Checking datasets

In [46]:
file_loc_cleaned = "../datasets/resale_hdb_price_coords_mrt_13jan.csv"
df_for_kaggle = pd.read_csv(
    file_loc_cleaned,
    parse_dates=["month", "lease_commence_date"],
    index_col=0,
    dtype={"x": "float64", "y": "float64", "postal": "object"},
    low_memory=False,
)

In [47]:
df_for_kaggle[df_for_kaggle.isna().any(axis=1)]
# df_for_kaggle.shape

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters


In [48]:
# df_for_kaggle = df_for_kaggle.drop([ "block", "street_name"], axis=1)
df_for_kaggle = df_for_kaggle.drop(["town", "block", "street_name"], axis=1)

In [49]:
today_date = dt.date.today().strftime("%Y-%d%b").lower()
kaggle_file_loc_updated_name = f"../datasets/resale_hdb_price_for_kaggle_{today_date}.csv"
df_for_kaggle.to_csv(kaggle_file_loc_updated_name)

In [50]:
df_for_kaggle

,month,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,2017-01-01,2 ROOM,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,473.6116,489.852867,406,ANG MO KIO AVENUE 10,NIL,560406,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1979-05-01,ANG MO KIO,NORTH-EAST REGION,30288.234663,38229.067463,1.362005,103.853880,Ang Mo Kio,999.941618,MRT,Red,8615.656983,TOWNSVILLE PRIMARY SCHOOL,218.125254
1,2017-01-01,3 ROOM,01 TO 03,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,721.1813,346.653470,108,ANG MO KIO AVENUE 4,KEBUN BARU HEIGHTS,560108,108 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SIN...,1978-08-01,ANG MO KIO,NORTH-EAST REGION,28543.458747,39220.009892,1.370966,103.838202,Mayflower,189.980291,MRT,Brown,9715.131951,ANG MO KIO PRIMARY SCHOOL,241.572335
2,2017-01-01,3 ROOM,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,721.1813,363.292836,602,ANG MO KIO AVENUE 5,YIO CHU KANG GREEN,560602,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1980-06-01,ANG MO KIO,NORTH-EAST REGION,28228.099954,40297.283149,1.380709,103.835368,Lentor,532.154773,MRT,Brown,10828.819556,ANDERSON PRIMARY SCHOOL,777.155378
3,2017-01-01,3 ROOM,04 TO 06,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,731.9452,362.048962,465,ANG MO KIO AVENUE 10,TECK GHEE HORIZON,560465,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,1980-02-01,ANG MO KIO,NORTH-EAST REGION,30657.824693,38693.098657,1.366201,103.857201,Ang Mo Kio,945.371842,MRT,Red,9097.929095,TECK GHEE PRIMARY SCHOOL,698.165530
4,2017-01-01,3 ROOM,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,721.1813,367.452678,601,ANG MO KIO AVENUE 5,YIO CHU KANG GREEN,560601,601 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1980-06-01,ANG MO KIO,NORTH-EAST REGION,28201.782245,40334.052030,1.381041,103.835132,Lentor,498.418205,MRT,Brown,10869.453109,ANDERSON PRIMARY SCHOOL,782.553222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197922,2025-01-01,5 ROOM,01 TO 03,122.0,Improved,1986-01-01,60 years 11 months,625000.0,1313.1958,475.938166,265,YISHUN STREET 22,NIL,760265,265 YISHUN STREET 22 SINGAPORE 760265,1986-12-01,YISHUN,NORTH REGION,28596.412191,46470.050392,1.436533,103.838678,Yishun,884.130863,MRT,Red,16912.389831,CHONGFU SCHOOL,217.599692
197923,2025-01-01,5 ROOM,10 TO 12,113.0,Improved,2017-01-01,91 years 09 months,730000.0,1216.3207,600.170662,511B,YISHUN STREET 51,OLEANDER BREEZE @ YISHUN,762511,511B YISHUN STREET 51 OLEANDER BREEZE @ YISHUN...,2017-09-01,YISHUN,NORTH REGION,29012.204603,44155.389938,1.415600,103.842414,Khatib,1068.249065,MRT,Red,14572.610297,NAVAL BASE PRIMARY SCHOOL,410.363918
197924,2025-01-01,5 ROOM,07 TO 09,122.0,Improved,1988-01-01,62 years 04 months,738000.0,1313.1958,561.987786,871,YISHUN STREET 81,NIL,760871,871 YISHUN STREET 81 SINGAPORE 760871,1988-03-01,YISHUN,NORTH REGION,28456.585910,43881.654344,1.413125,103.837421,Khatib,682.691361,MRT,Red,14349.375919,NAVAL BASE PRIMARY SCHOOL,383.583811
197925,2025-01-01,5 ROOM,10 TO 12,127.0,Improved,1988-01-01,62 years 04 months,740000.0,1367.0153,541.325324,851,YISHUN STREET 81,KHATIB EVERGREEN I,760851,851 YISHUN STREET 81 KHATIB EVERGREEN I SINGAP...,1988-03-01,YISHUN,NORTH REGION,28388.618487,44154.015015,1.415588,103.836811,Khatib,470.278090,MRT,Red,14627.567404,NAVAL BASE PRIMARY SCHOOL,234.604531
